In [1]:
import numpy as np
from keras.datasets import mnist

In [2]:
# with open('fashion_mnist_train.npy', 'rb') as train_data:
#     x_train = np.load(train_data)
#     y_train = np.load(train_data)

# with open ('fashion_mnist_test.npy', 'rb') as test_data:
#     x_test = np.load(test_data)
#     y_test = np.load(test_data)

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train.shape

(60000, 28, 28)

In [5]:
y_train.shape

(60000,)

In [6]:
x_test.shape

(10000, 28, 28)

In [7]:
y_test.shape

(10000,)

In [8]:
x_train = x_train[:10000]
y_train = y_train[:10000]

x_test = x_test[:10000]
y_test = y_test[:10000]

In [9]:
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0

In [10]:
x_test = x_test.reshape(x_test.shape[0],-1) / 255.0

In [11]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [12]:
class NN:
    def __init__(self, input_neurons, hidden_neurons, output_neurons, lr, e):
        self.input_neurons = input_neurons
        self.hidden_neurons = hidden_neurons
        self.output_neurons = output_neurons
        self.e = e

        #links of weights from input to hidden neurons
        self.wih = np.random.normal(0.0, pow(self.input_neurons, -0.5), (self.hidden_neurons, self.input_neurons))
        self.bih = 0
        
        #links of weights from hidden to output neurons
        self.who = np.random.normal(0.0, pow(self.hidden_neurons, -0.5),(self.output_neurons, self.hidden_neurons))
        self.bho = 0

        self.lr = lr
    def activation(self, z):
        z = np.clip(z,-500,500)
        return 1 / (1 + np.exp (-z)) #sigmoid
        #return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z)) #tanh

    def activation_derivative(self,z):
        return self.activation(z) * (1 - self.activation(z)) #sigmoid
        #return 1-self.activation(z)**2 #tanh
        
    def forward(self, input_list):
        inputs = np.array(input_list, ndmin = 2).T
        #passing inputs to hidden layer
        hidden_inputs = np.dot(self.wih,inputs) + self.bih
        #getting outputs from hidden layer
        hidden_outputs = self.activation(hidden_inputs)
        #passing outputs of hidden layer to output layer as input
        output_inputs = np.dot(self.who, hidden_outputs) + self.bho
        #getting ouputs from the output layer
        output_outputs = self.activation(output_inputs)

        return output_outputs

    def backprop(self, inputs_list, targets_list):
        inputs = np.array(inputs_list,ndmin = 2).T
        tj = np.array(targets_list, ndmin = 2).T
        #passing inputs to hidden layer
        hidden_inputs = np.dot(self.wih, inputs) + self.bih
        #getting outputs from hidden layer
        hidden_outputs = self.activation(hidden_inputs)
        #passing inputs from hidden layer outputs to output layer
        output_inputs = np.dot(self.who, hidden_outputs) + self.bho
        #getting output from the output layer
        yj = self.activation(output_inputs)
        #finding the errors from the output layer
        output_errors = -(tj - yj)
        #finding the errors in the hidden layer
        hidden_errors = np.dot(self.who.T, output_errors)

        # #updating the weights using Gradient Desctent
        # self.who -= self.lr * np.dot(self.activation_derivative(output_inputs), np.transpose(hidden_outputs))
        # #self.wih -= self.lr * np.dot(np.dot(self.who.T,(self.activation_derivative(output_inputs)*(np.dot(self.who,self.activation_derivative(hidden_inputs)))),np.transpose(inputs))
        # self.wih -= self.lr * np.dot((self.activation_derivative(hidden_inputs)* np.dot(self.who.T, self.activation_derivative(output_inputs))), np.transpose(inputs))
        # self.bho -= self.lr * (self.activation_derivative(output_inputs))
        # self.bih -= self.lr * np.dot(self.who.T, self.activation_derivative(output_inputs))* self.activation_derivative(hidden_inputs)
        
        self.who -= self.lr * np.dot((output_errors * self.activation_derivative(yj)), np.transpose(hidden_outputs))
        self.wih -= self.lr * np.dot((hidden_errors * self.activation_derivative(hidden_outputs)), np.transpose(inputs))


        #updating bias
        self.bho -= self.lr * (output_errors * self.activation_derivative(yj))
        self.bih -= self.lr * (hidden_errors * self.activation_derivative(hidden_outputs))
        
        pass
        
    def fit(self, inputs_list, targets_list):
        for e in range(self.e):
            self.backprop(inputs_list, targets_list)
            print(f"Epoch{e}/{self.e}completed.")

    def predict(self, X):
        outputs = self.forward(X).T
        return outputs

In [13]:
nn = NN(input_neurons=784, hidden_neurons= 184, output_neurons=10, lr=0.01, e= 100)

In [14]:
nn.fit(x_train, y_train)

Epoch0/100completed.
Epoch1/100completed.
Epoch2/100completed.
Epoch3/100completed.
Epoch4/100completed.
Epoch5/100completed.
Epoch6/100completed.
Epoch7/100completed.
Epoch8/100completed.
Epoch9/100completed.
Epoch10/100completed.
Epoch11/100completed.
Epoch12/100completed.
Epoch13/100completed.
Epoch14/100completed.
Epoch15/100completed.
Epoch16/100completed.
Epoch17/100completed.
Epoch18/100completed.
Epoch19/100completed.
Epoch20/100completed.
Epoch21/100completed.
Epoch22/100completed.
Epoch23/100completed.
Epoch24/100completed.
Epoch25/100completed.
Epoch26/100completed.
Epoch27/100completed.
Epoch28/100completed.
Epoch29/100completed.
Epoch30/100completed.
Epoch31/100completed.
Epoch32/100completed.
Epoch33/100completed.
Epoch34/100completed.
Epoch35/100completed.
Epoch36/100completed.
Epoch37/100completed.
Epoch38/100completed.
Epoch39/100completed.
Epoch40/100completed.
Epoch41/100completed.
Epoch42/100completed.
Epoch43/100completed.
Epoch44/100completed.
Epoch45/100completed

In [15]:
probs = nn.predict(x_test)

predictions = []

for prob in probs:
    max_idx = np.argmax(prob)
    prediction = np.zeros_like(prob)
    prediction[max_idx] = 1
    predictions.append(prediction)

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print("Accuracy: ", accuracy_score(predictions, y_test))
print("CR: ", classification_report(predictions, y_test))

Accuracy:  0.7835
CR:                precision    recall  f1-score   support

           0       0.83      0.93      0.88       870
           1       0.92      0.97      0.94      1078
           2       0.80      0.84      0.82       985
           3       0.77      0.81      0.79       962
           4       0.50      0.83      0.62       588
           5       0.84      0.54      0.66      1402
           6       0.79      0.92      0.85       831
           7       0.77      0.94      0.85       850
           8       0.73      0.74      0.74       959
           9       0.85      0.58      0.69      1475

   micro avg       0.78      0.78      0.78     10000
   macro avg       0.78      0.81      0.78     10000
weighted avg       0.80      0.78      0.78     10000
 samples avg       0.78      0.78      0.78     10000



In [17]:
import tensorflow as tf
print("GPU Available: ", tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available:  False


Python 3.11.3
